In [0]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://us.mirrors.quenda.co/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# !tar xf spark-2.4.5-bin-hadoop2.7.tgz
# !pip install -q findspark

In [0]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [0]:
# from google.colab import files
# files.upload()

MessageError: ignored

In [0]:
from pyspark.sql import *
import pyspark.sql.functions as f
from pyspark.ml.feature import Word2Vec, CountVectorizer, CountVectorizerModel, StopWordsRemover
from pyspark.sql.types import FloatType, IntegerType
from pyspark.ml.classification import LogisticRegression

In [0]:
spark = SparkSession.builder.appName('DicAssign3').config("spark.some.config.option","some-value")\
        .config("spark.executor.memory", "16g")\
        .config("spark.driver.memory", "16g")\
        .config("spark.memory.offHeap.enabled",'true')\
        .config("spark.memory.offHeap.size","8g")\
        .getOrCreate()

In [0]:
df = spark.read.csv("train.csv",inferSchema="true",header="true",escape='"')
df = df.na.drop(subset=["genre"])
df_test = spark.read.csv("test.csv",inferSchema="true",header="true",escape='"')
mappings = spark.read.csv("mapping.csv",header="true")

In [0]:
mappings = mappings.withColumn("genre_array", f.split(f.col("0"), ",\s*"))
mapping_list = ["Drama","Comedy","Romance Film","Thriller","Action","World cinema","Crime Fiction","Horror","Black-and-white","Indie","Action/Adventure","Adventure","Family Film","Short Film","Romantic drama","Animation","Musical","Science Fiction","Mystery","Romantic comedy"]

In [0]:
df1 = df.withColumn("genre_array", f.regexp_replace(f.col("genre"), r'[\[\]\']', ''))

In [0]:
df2 = df1.withColumn("genre_array", f.split(f.col("genre_array"), ", "))

In [0]:
# vect = CountVectorizer(inputCol="genre_array",outputCol="ohe_genre",vocabSize=20)
vect = CountVectorizerModel.from_vocabulary(vocabulary=mapping_list,inputCol="genre_array",outputCol="ohe_genre")
# vect_model = vect.fit(df2)
df_ohe = vect.transform(df2)

In [0]:
# clean plot
df_ohe = df_ohe.withColumn("plot", f.regexp_replace(f.col("plot"), r'[^A-Za-z ]', ''))
df_ohe = df_ohe.withColumn("plot", f.regexp_replace(f.col("plot"), '\s+', ' '))
df_ohe = df_ohe.withColumn("plot", f.lower(f.col("plot")))

In [0]:
# clean test plot
mappins_ohe_test = df_test.withColumn("plot", f.regexp_replace(f.col("plot"), r'[^A-Za-z ]', ''))
mappins_ohe_test = mappins_ohe_test.withColumn("plot", f.regexp_replace(f.col("plot"), '\s+', ' '))
mappins_ohe_test = mappins_ohe_test.withColumn("plot", f.lower(f.col("plot")))

In [0]:
df_ohe1 = df_ohe.withColumn("plot_tokens", f.split(f.col("plot"), " "))
mappins_ohe_test1 = mappins_ohe_test.withColumn("plot_tokens", f.split(f.col("plot"), " "))

In [0]:
remover = StopWordsRemover(inputCol="plot_tokens", outputCol="filtered_plot")
filtered_df = remover.transform(df_ohe1)
filtered_df_test = remover.transform(mappins_ohe_test1)

In [0]:
word2VecEstimator = Word2Vec(vectorSize=450, minCount=0, inputCol="filtered_plot", outputCol="matrix")
model = word2VecEstimator.fit(filtered_df)

result = model.transform(filtered_df)
result_test = model.transform(filtered_df_test)

In [16]:
result.select("matrix").head(1)

[Row(matrix=DenseVector([-0.0318, 0.0205, 0.0, 0.0418, -0.0785, -0.0081, 0.0635, -0.0493, 0.0318, -0.015, -0.0263, -0.0083, 0.003, 0.0212, 0.0062, 0.0347, -0.0001, -0.0504, -0.0045, -0.0511, 0.0599, -0.0081, 0.0432, 0.04, 0.0011, -0.002, 0.0284, 0.031, 0.0067, 0.0085, 0.0124, 0.0463, 0.0596, -0.0028, 0.0027, -0.0338, 0.0231, -0.0281, 0.0179, 0.0316, 0.0002, -0.0565, 0.0278, 0.014, -0.0566, -0.0077, 0.0222, -0.0316, 0.0197, -0.0241, -0.0197, 0.0515, -0.0342, 0.0127, -0.0739, -0.0051, 0.0269, -0.0304, -0.0017, 0.015, -0.0341, 0.0058, -0.0314, -0.0033, -0.035, 0.0056, 0.049, 0.0116, -0.0256, 0.0282, 0.0192, 0.0011, -0.0048, -0.0342, 0.0243, -0.0031, 0.0091, 0.0006, -0.0229, -0.0128, -0.0184, 0.0365, 0.009, 0.0084, -0.025, -0.006, 0.0095, -0.012, 0.0071, 0.0384, 0.0301, 0.0439, 0.0394, 0.0138, 0.0241, 0.0199, 0.0066, 0.0432, -0.0186, -0.0218, -0.0098, 0.0271, -0.0502, -0.0211, 0.0304, -0.0412, -0.0323, 0.029, -0.0846, -0.065, -0.0003, -0.0479, 0.0075, -0.017, 0.0144, 0.0136, -0.0158, -0.01

In [17]:
word2Vec = result.selectExpr("movie_id","matrix as features", "ohe_genre as label")
word2Vec_test = result_test.selectExpr("movie_id","matrix as features")
word2Vec.count()

31109

In [0]:
lrModels = []
for i in range(20):
    firstElement = f.udf(lambda v: float(v[i]), FloatType())

    new_df = word2Vec.withColumn("label", firstElement("label"))
    lr = LogisticRegression(maxIter=100, regParam=0.3, elasticNetParam=0)
    lrModel = lr.fit(new_df)
    lrModels.append(lrModel)

In [0]:
predictions = []
for i in range(len(lrModels)):
    firstElement = f.udf(lambda v: float(v[i]), FloatType())

    new_df = word2Vec.withColumn("label", firstElement("label"))
    predicted = lrModels[i].transform(word2Vec_test)
    predicted = predicted.withColumn("prediction", predicted["prediction"].cast(IntegerType()))
    predictions.append(predicted)

In [0]:
final_df = predictions[0].selectExpr("movie_id","prediction as predictions")
for i in range(1,len(predictions)):
    final_df = final_df.join(predictions[i].selectExpr("movie_id","prediction as prediction"+str(i)), ["movie_id"])
    final_df = final_df.withColumn("predictions", f.concat(f.col("predictions"), f.lit(" "), f.col("prediction"+str(i)))).select("movie_id","predictions")

In [21]:
final_df.show(truncate=False)

+--------+---------------------------------------+
|movie_id|predictions                            |
+--------+---------------------------------------+
|1335380 |1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|29062594|1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|9252321 |1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|13455076|0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|24165951|0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|1925869 |1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|10799612|1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|28238240|0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0|
|17124781|0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|28207941|0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0|
|19174305|1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|18392317|0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|34420857|1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|4039635 |0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|8034072 |1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|4016437 |1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|1520023 |0 0 0 0 0 0 0 0 0 0 0

In [0]:
final_df.toPandas().to_csv("pred3.csv",index = False)